In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
path = "E:/Research/Ovarian Cancer/Articles/Orginal/"
data = pd.read_excel(path+"Data.xlsx", sheet_name="Sheet1.5")

In [4]:
data.head(5)

,Age,ProvinceName,FReportDate,C_code,C_subcode,M_code,B_code,Grading,Date.of.death,Reason.of.death,Alive.Date.of.last.follow.up,Year,Status,Survt,Survt.Year
0,68.139726,نامعلوم,1390/02/05,56,9,8010,3,9.0,1395/08/19,سرطان,NaN,1390,1.0,2025.0,5.547945
1,34.906849,نامعلوم,1390/11/18,56,9,8441,3,2.0,NaN,NaN,NaN,1390,NaN,NaN,NaN
2,60.726027,نامعلوم,1390/09/06,56,9,8620,3,9.0,NaN,NaN,NaN,1390,NaN,NaN,NaN
3,25.794521,نامعلوم,1390/10/09,56,9,9060,3,9.0,NaN,NaN,1398/10/21,1390,0.0,2934.0,8.038356
4,50.260274,نامعلوم,1390/03/22,56,9,8140,3,9.0,NaN,NaN,NaN,1390,NaN,NaN,NaN


In [5]:
data.shape

(7977, 15)

In [10]:
path = "E:/Research/Ovarian Cancer/Articles/Orginal/"
data = pd.read_excel(path+"data_ready2.xlsx")

In [15]:
data.MonthSurvive.max()

120

In [16]:
# Impute Age < 10

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['Histology'] = le.fit_transform(data.Histology.values)
data['Type'] = le.fit_transform(data.Type.values)


from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
data = imputer.fit_transform(data)

data = pd.DataFrame(data, columns=['Age', 'Grading', 'Histopathology', 'Topography', 'Status', 'MonthSurvive', 'YearSurvive'])

In [7]:
# fill lost survival date and time

#data.isna().sum()

# create time column with any date to 1397.12.30

from datetime import datetime

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month


# calculate month for death
for index, date in enumerate(zip(data['FReportDate'], data['DeathDate'])):
    if pd.notna(date[1]):
        month_report = datetime.strptime(str(date[0][:7]) , '%Y/%m')
        month_death = datetime.strptime(str(date[1][:7]), '%Y/%m')
        data.loc[index, 'MonthSurvive'] = diff_month(month_death, month_report)

# calculate for alive
for index, date in enumerate(zip(data['FReportDate'], data['AliveDate'])):
    if pd.notna(date[1]):
        month_report = datetime.strptime(str(date[0][:7]) , '%Y/%m')
        month_alive = datetime.strptime(str(date[1][:7]), '%Y/%m')
        data.loc[index, 'MonthSurvive'] = diff_month(month_alive, month_report)

In [8]:
# Ovarian cancer histology types and groups

type_1 = {
    'Clear cell carcinoma': [8005, 8310, 8443, 9110],
    'Endometrioid carcinoma': [8380, 8382, 8383, 8560, 8570],
    'Mucinous carcinoma': [8470, 8471, 8480, 8481, 8482, 8490],
    'Squamous carcinoma': [8051,8052,8053,8054,8055,8056,8057,8058,8059,8060,8061,8062,8063,8064,8065,8066,8067,8068,8069,8070,8071,8072,8073,8074,8075,8076,8077,8078,8079,8080,8081,8082,8083,8084],
    'Transitional cell': [8120,8121,8122,8123,8124,8125,8126,8127,8128,8129,8130,8131,9000]
}

type_2 = {
    'Serous carcinoma': [8050, 8441, 8450, 8460, 8461],
    'Mixed epithelial-stromal carcinoma': [8313, 8323, 8381, 8930, 8931, 8932, 8933, 8934, 8935, 8936, 8937, 8938, 8939, 8940, 8941, 8942, 8943, 8944, 8945, 8946, 8947, 8948, 8949, 8950, 8951, 8952, 8953, 8954, 8955, 8956, 8957, 8958, 8959, 8960, 8961, 8962, 8963, 8964, 8965, 8966, 8967, 8968, 8969, 8970, 8971, 8972, 8973, 8974, 8975, 8976, 8977, 8978, 8979, 8980, 8981, 8982, 8983, 8984, 8985, 8986, 8987, 8988, 8989, 8990, 8991, 9010, 9011,9012,9013,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023,9024,9025,9026,9027,9028,9029,9030],
    'Undifferentiated': [8010,8011,8012,8013,8014,8015, 8020,8021,8022,8023,8024,8025,8026,8027,8028,8029,8030,8031,8032,8033,8034,8035,8036,8037,8038,8039,8040,8041,8042,8043,8044,8045,8046, 8090,8091,8092,8093,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106,8107,8108,8109,8110, 8140, 8141, 8142, 8143, 8144, 8145, 8146, 8147, 8148, 8149, 8150, 8151, 8152, 8153, 8154, 8155, 8156, 8157, 8158, 8159, 8160, 8161, 8162, 8163, 8164, 8165, 8166, 8167, 8168, 8169, 8170, 8171, 8172, 8173, 8174, 8175, 8176, 8177, 8178, 8179, 8180, 8181, 8182, 8183, 8184, 8185, 8186, 8187, 8188, 8189, 8190, 8191, 8192, 8193, 8194, 8195, 8196, 8197, 8198, 8199, 8200, 8201, 8202, 8203, 8204, 8205, 8206, 8207, 8208, 8209, 8210, 8211, 8212, 8213, 8214, 8215, 8216, 8217, 8218, 8219, 8220, 8221, 8222, 8223, 8224, 8225, 8226, 8227, 8228, 8229, 8230, 8231, 8246, 8247, 8248, 8249, 8250, 8251, 8252, 8253, 8254, 8255, 8256, 8257, 8258, 8259, 8260, 8261, 8262, 8263, 8264, 8265, 8266, 8267, 8268, 8269, 8270, 8271, 8272, 8273, 8274, 8275, 8276, 8277, 8278, 8279, 8280, 8281, 8282, 8283, 8284, 8285, 8286, 8287, 8288, 8289, 8290, 8291, 8292, 8293, 8294, 8295, 8296, 8297, 8298, 8299, 8300, 8311,8312, 8314, 8315, 8316, 8317, 8318, 8319, 8320, 8321, 8322, 8324,8325, 8336,8337, 8341, 8342, 8343, 8344, 8345, 8346, 8347, 8348, 8349, 8350, 8351, 8352, 8353, 8354, 8355, 8356, 8357, 8358, 8359, 8360, 8361, 8362, 8363, 8364, 8365, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8374, 8375, 8384, 8385, 8386, 8387, 8388, 8389, 8390, 8391, 8392, 8393, 8394, 8395, 8396, 8397, 8398, 8399, 8400, 8401, 8402, 8403, 8404, 8405, 8406, 8407, 8408, 8409, 8410, 8411, 8412, 8413, 8414, 8415, 8416, 8417, 8418, 8419, 8420, 8421, 8422, 8423, 8424, 8425, 8426, 8427, 8428, 8429, 8430, 8431, 8432, 8433, 8434, 8435, 8436, 8437, 8438, 8439, 8440, 8452, 8453, 8454, 8500, 8501, 8502, 8503, 8504, 8505, 8506, 8507, 8508, 8509, 8510, 8511, 8512, 8513, 8514, 8515, 8516, 8517, 8518, 8519, 8520, 8521, 8522, 8523, 8524, 8525, 8526, 8527, 8528, 8529, 8530, 8531, 8532, 8533, 8534, 8535, 8536, 8537, 8538, 8539, 8540, 8541, 8542, 8543, 8544, 8545, 8546, 8547, 8548, 8549, 8550, 8551, 8561, 8562, 8571, 8572, 8573, 8574, 8575, 8576, 8577, 8578, 8579, 8580, 8581, 8582, 8583, 8584, 8585, 8586, 8587, 8588, 8589],
}

other = {
    'Germ cell':[8240, 8241, 8242, 8243, 8244, 8245, 8330, 8331, 8332, 8333, 8334, 8335, 8340, 9060, 9061, 9062, 9063, 9064, 9065, 9066, 9067, 9068, 9069, 9070, 9071, 9072, 9073, 9074, 9075, 9076, 9077, 9078, 9079, 9080, 9081, 9082, 9083, 9084, 9085, 9086, 9087, 9088, 9089, 9090, 9091, 9092, 9093, 9094, 9095, 9096, 9097, 9098, 9099, 9100, 9101, 9102, 9103, 9104, 9380, 9381, 9382, 9383, 9384, 9385, 9386, 9387, 9388, 9389, 9390, 9391, 9392, 9393, 9394, 9395, 9396, 9397, 9398, 9399, 9400, 9401, 9402, 9403, 9404, 9405, 9406, 9407, 9408, 9409, 9410, 9411, 9412, 9413, 9414, 9415, 9416, 9417, 9418, 9419, 9420, 9421, 9422, 9423, 9424, 9425, 9426, 9427, 9428, 9429, 9430, 9431, 9432, 9433, 9434, 9435, 9436, 9437, 9438, 9439, 9440, 9441, 9442, 9443, 9444, 9445, 9446, 9447, 9448, 9449, 9450, 9451, 9452, 9453, 9454, 9455, 9456, 9457, 9458, 9459, 9460, 9461, 9462, 9463, 9464, 9465, 9466, 9467, 9468, 9469, 9470, 9471, 9472, 9473, 9474, 9475, 9476, 9477, 9478, 9479, 9480, 9481, 9482, 9483, 9484, 9485, 9486, 9487, 9488, 9489, 9490, 9491, 9492, 9493, 9494, 9495, 9496, 9497, 9498, 9499, 9500, 9501, 9502, 9503, 9504, 9505, 9506, 9507, 9508, 9509, 9510, 9511, 9512, 9513, 9514, 9515, 9516, 9517, 9518, 9519, 9520, 9521, 9522, 9523],
    'Sex cord-stromal':[8590, 8591, 8592, 8593, 8594, 8595, 8596, 8597, 8598, 8599, 8600, 8601, 8602, 8603, 8604, 8605, 8606, 8607, 8608, 8609, 8610, 8611, 8612, 8613, 8614, 8615, 8616, 8617, 8618, 8619, 8620, 8621, 8622, 8623, 8624, 8625, 8626, 8627, 8628, 8629, 8630, 8631, 8632, 8633, 8634, 8635, 8636, 8637, 8638, 8639, 8640, 8641, 8642, 8643, 8644, 8645, 8646, 8647, 8648, 8649, 8650, 8651, 8652, 8653, 8654, 8655, 8656, 8657, 8658, 8659, 8660, 8661, 8662, 8663, 8664, 8665, 8666, 8667, 8668, 8669, 8670, 8671, 8810],
    'Other specific non-epithelial':[8680, 8681, 8682, 8683, 8684, 8685, 8686, 8687, 8688, 8689, 8690, 8691, 8692, 8693, 8694, 8695, 8696, 8697, 8698, 8699, 8700, 8701, 8702, 8703, 8704, 8705, 8706, 8707, 8708, 8709, 8710, 8711, 8712, 8713, 8714, 8715, 8716, 8717, 8718, 8719, 8720, 8721, 8722, 8723, 8724, 8725, 8726, 8727, 8728, 8729, 8730, 8731, 8732, 8733, 8734, 8735, 8736, 8737, 8738, 8739, 8740, 8741, 8742, 8743, 8744, 8745, 8746, 8747, 8748, 8749, 8750, 8751, 8752, 8753, 8754, 8755, 8756, 8757, 8758, 8759, 8760, 8761, 8762, 8763, 8764, 8765, 8766, 8767, 8768, 8769, 8770, 8771, 8772, 8773, 8774, 8775, 8776, 8777, 8778, 8779, 8780, 8781, 8782, 8783, 8784, 8785, 8786, 8787, 8788, 8789, 8790, 8791, 8792, 8793, 8794, 8795, 8796, 8797, 8798, 8799, 8800, 8801, 8802, 8803, 8804, 8805, 8806, 8811, 8812, 8813, 8814, 8815, 8816, 8817, 8818, 8819, 8820, 8821, 8822, 8823, 8824, 8825, 8826, 8827, 8828, 8829, 8830, 8831, 8832, 8833, 8834, 8835, 8836, 8837, 8838, 8839, 8840, 8841, 8842, 8843, 8844, 8845, 8846, 8847, 8848, 8849, 8850, 8851, 8852, 8853, 8854, 8855, 8856, 8857, 8858, 8859, 8860, 8861, 8862, 8863, 8864, 8865, 8866, 8867, 8868, 8869, 8870, 8871, 8872, 8873, 8874, 8875, 8876, 8877, 8878, 8879, 8880, 8881, 8882, 8883, 8884, 8885, 8886, 8887, 8888, 8889, 8890, 8891, 8892, 8893, 8894, 8895, 8896, 8897, 8898, 8899, 8900, 8901, 8902, 8903, 8904, 8905, 8906, 8907, 8908, 8909, 8910, 8911, 8912, 8913, 8914, 8915, 8916, 8917, 8918, 8919, 8920, 8921, 9040, 9041, 9042, 9043, 9044, 9045, 9046, 9047, 9048, 9049, 9050, 9051, 9052, 9053, 9054, 9120, 9121, 9122, 9123, 9124, 9125, 9126, 9127, 9128, 9129, 9130, 9131, 9132, 9133, 9134, 9135, 9136, 9137, 9138, 9139, 9140, 9141, 9142, 9143, 9144, 9145, 9146, 9147, 9148, 9149, 9150, 9151, 9152, 9153, 9154, 9155, 9156, 9157, 9158, 9159, 9160, 9161, 9162, 9163, 9164, 9165, 9166, 9167, 9168, 9169, 9170, 9171, 9172, 9173, 9174, 9175, 9176, 9177, 9178, 9179, 9180, 9181, 9182, 9183, 9184, 9185, 9186, 9187, 9188, 9189, 9190, 9191, 9192, 9193, 9194, 9195, 9196, 9197, 9198, 9199, 9200, 9201, 9202, 9203, 9204, 9205, 9206, 9207, 9208, 9209, 9210, 9211, 9212, 9213, 9214, 9215, 9216, 9217, 9218, 9219, 9220, 9221, 9222, 9223, 9224, 9225, 9226, 9227, 9228, 9229, 9230, 9231, 9232, 9233, 9234, 9235, 9236, 9237, 9238, 9239, 9240, 9241, 9242, 9243, 9244, 9245, 9246, 9247, 9248, 9249, 9250, 9251, 9252, 9253, 9254, 9255, 9256, 9257, 9258, 9259, 9260, 9261, 9262, 9263, 9264, 9265, 9266, 9267, 9268, 9269, 9270, 9271, 9272, 9273, 9274, 9275, 9276, 9277, 9278, 9279, 9280, 9281, 9282, 9283, 9284, 9285, 9286, 9287, 9288, 9289, 9290, 9291, 9292, 9293, 9294, 9295, 9296, 9297, 9298, 9299, 9300, 9301, 9302, 9303, 9304, 9305, 9306, 9307, 9308, 9309, 9310, 9311, 9312, 9313, 9314, 9315, 9316, 9317, 9318, 9319, 9320, 9321, 9322, 9323, 9324, 9325, 9326, 9327, 9328, 9329, 9330, 9331, 9332, 9333, 9334, 9335, 9336, 9337, 9338, 9339, 9340, 9341, 9342, 9343, 9344, 9345, 9346, 9347, 9348, 9349, 9350, 9351, 9352, 9353, 9354, 9355, 9356, 9357, 9358, 9359, 9360, 9361, 9362, 9363, 9364, 9365, 9366, 9367, 9368, 9369, 9370, 9371, 9372, 9373, 9530, 9531, 9532, 9533, 9534, 9535, 9536, 9537, 9538, 9539, 9540, 9541, 9542, 9543, 9544, 9545, 9546, 9547, 9548, 9549, 9550, 9551, 9552, 9553, 9554, 9555, 9556, 9557, 9558, 9559, 9560, 9561, 9562, 9563, 9564, 9565, 9566, 9567, 9568, 9569, 9570, 9571, 9572, 9573, 9574, 9575, 9576, 9577, 9578, 9579, 9580, 9581, 9582, 9583, 9584, 9585, 9586, 9587, 9588, 9589],
    'Non-specific':[8000,8001,8002,8003,8004]
}

In [9]:
# calculate Histology for each patient with hers codes

for index, code in enumerate(data['Mcode']):
    if pd.notna(code):
        for i in [type_1, type_2, other]:
            for key, value in i.items():
                if code in value:
                    data.loc[index, 'Histology'] = key
                    if i in [type_1]:
                        data.loc[index, 'Type'] = 'type_1'
                    elif i in [type_2]:
                        data.loc[index, 'Type'] = 'type_2'
                    else:
                        data.loc[index, 'Type'] = key     

                        
# Calculate patient survival by YEAR

for i, x in enumerate(data['MonthSurvive']):
    if 108 < x <= 120:
        data.loc[i ,'YearSurvive'] = 10
    elif 96 < x <= 108:
        data.loc[i, 'YearSurvive'] = 9
    elif 84 < x <= 96:
        data.loc[i, 'YearSurvive'] = 8
    elif 72 < x <= 84:
        data.loc[i, 'YearSurvive'] = 7
    elif 60 < x <= 72:
        data.loc[i, 'YearSurvive'] = 6
    elif 48 < x <= 60:
        data.loc[i ,'YearSurvive'] = 5
    elif 36 < x <= 48:
        data.loc[i, 'YearSurvive'] = 4
    elif 24 < x <= 36:
        data.loc[i, 'YearSurvive'] = 3
    elif 12 < x <= 24:
        data.loc[i, 'YearSurvive'] = 2
    elif x <= 12:
        data.loc[i, 'YearSurvive'] = 1


In [10]:
# delete all rows with column 'MonthSurvive' below 'Zero' OR 'DeathValid' equel 2
IndexRow = data[ (data['MonthSurvive'] < 0) | (data['DeathValid'] == 2) ].index
data.drop(IndexRow , inplace=True)

In [11]:
data.head()

,Age,ProvinceName,FReportDate,Mcode,Grading,DeathDate,DeathValid,AliveDate,AliveValid,Year,Status,SurvT,SurvY,MonthSurvive,Histology,Type,YearSurvive
71,49.0,تهران,1393/08/01,8471,9.0,NaN,0,1398/04/10,1,1393,0.0,1712.0,4.690411,56.0,Mucinous carcinoma,type_1,5.0
72,23.0,خراسان رضوی,1393/02/31,9060,9.0,NaN,0,1398/02/05,1,1393,0.0,1800.0,4.931507,60.0,Germ cell,Germ cell,5.0
73,61.0,خراسان رضوی,1393/12/14,8980,3.0,NaN,0,1399/02/04,1,1393,0.0,1876.0,5.139726,62.0,Mixed epithelial-stromal carcinoma,type_2,6.0
74,47.0,همدان,1393/12/12,9064,9.0,NaN,0,1399/02/03,1,1393,0.0,1877.0,5.142466,62.0,Germ cell,Germ cell,6.0
75,52.0,زنجان,1393/12/29,8000,9.0,NaN,0,1399/04/02,1,1393,0.0,1921.0,5.263014,64.0,Non-specific,Non-specific,6.0


In [13]:
data.Grading.value_counts()

9.0     1877
3.0      304
2.0      279
1.0      273
4.0       35
0.0        8
6.0        2
39.0       1
Name: Grading, dtype: int64

In [11]:
# Seperate important features for Classificatio Modeling

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

X = data[['Age', 'Grading', 'Histology', 'Type']]
Y = le.fit_transform(data['YearSurvive'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state = 42, shuffle = True)

# cancer type, Grading and Histology label encoding for train
#X_train.Type = le.fit_transform(X_train.Type)
X_train.Histology = le.fit_transform(X_train.Histology)
X_train.Grading = le.fit_transform(X_train.Grading)

# cancer type, Grading and Histology label encoding for test
#X_test.Type = le.fit_transform(X_test.Type)
X_test.Histology = le.fit_transform(X_test.Histology)
X_test.Grading = le.fit_transform(X_test.Grading)

In [12]:
# check missed data

def check_missing(data,output_path=None):
    """
    check the total number & percentage of missing values
    per variable of a pandas Dataframe
    """
    
    result = pd.concat([data.isnull().sum(),data.isnull().mean()],axis=1)
    result = result.rename(index=str,columns={0:'total missing',1:'proportion'})
    if output_path is not None:
        result.to_csv(output_path+'missing.csv')
        print('result saved at', output_path, 'missing.csv')
    return result

check_missing(data=pd.DataFrame(X_train))

,total missing,proportion
Age,0,0.0
Grading,0,0.0
Histology,0,0.0
Type,0,0.0


# IMPUTE CLEANED DATA

In [43]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn_pandas import DataFrameMapper

path = "E:/Research/
data = pd.read_excel(path+"data_ready.xlsx", sheet_name='Sheet2')

data.Age = data.Age.apply(lambda x: int(x) if not pd.isna(x) else x)

In [44]:
data.head()

,Age,Grading,Histology,Type,Status,MonthSurvive,YearSurvive
0,59.0,1.0,Endometrioid carcinoma,type_1,1,1,1
1,NaN,2.0,Serous carcinoma,type_2,1,1,1
2,33.0,2.0,Undifferentiated,type_2,1,1,1
3,33.0,2.0,Undifferentiated,type_2,1,1,1
4,64.0,2.0,Serous carcinoma,type_2,1,1,1


In [45]:
data[['Grading', 'Histology', 'Type']] = data[['Grading', 'Histology', 'Type']].astype('category').apply(lambda x: x.cat.codes)

In [46]:
data

,Age,Grading,Histology,Type,Status,MonthSurvive,YearSurvive
0,59.0,0,1,4,1,1,1
1,NaN,1,7,5,1,1,1
2,33.0,1,11,5,1,1,1
3,33.0,1,11,5,1,1,1
4,64.0,1,7,5,1,1,1
...,...,...,...,...,...,...,...
2670,44.0,4,7,5,0,120,10
2671,46.0,4,2,0,0,120,10
2672,48.0,4,5,1,0,120,10
2673,53.0,4,7,5,0,120,10


In [47]:
data.isna().sum()

Age             19
Grading          0
Histology        0
Type             0
Status           0
MonthSurvive     0
YearSurvive      0
dtype: int64

In [48]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
data = imputer.fit_transform(data)

In [49]:
df_data = pd.DataFrame(data, columns=["Age", "Grading", "histology", "Type", 'Status', 'MonthSurvive', 'YearSurvive'])

In [50]:
df_data

,Age,Grading,histology,Type,Status,MonthSurvive,YearSurvive
0,59.0,0.0,1.0,4.0,1.0,1.0,1.0
1,57.6,1.0,7.0,5.0,1.0,1.0,1.0
2,33.0,1.0,11.0,5.0,1.0,1.0,1.0
3,33.0,1.0,11.0,5.0,1.0,1.0,1.0
4,64.0,1.0,7.0,5.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
2670,44.0,4.0,7.0,5.0,0.0,120.0,10.0
2671,46.0,4.0,2.0,0.0,0.0,120.0,10.0
2672,48.0,4.0,5.0,1.0,0.0,120.0,10.0
2673,53.0,4.0,7.0,5.0,0.0,120.0,10.0


In [51]:
df_data.isna().sum()

Age             0
Grading         0
histology       0
Type            0
Status          0
MonthSurvive    0
YearSurvive     0
dtype: int64